# IPSC Project
## Fast Training of Convolutional Networks Through FFTs

### Team Members
- Pragun Saxena (20171127)
- Shivam Bansal (20171001)
- Abhinav Vaishya (2018121003)

# Using Cuda in python 
- Remember to use cuda you need to have runtime as GPU everytime(you can change it in settings above).
- the below code installs extension that are needed for running cuda along with python in notebook
- Whenever you reopen the notebook, run cell 4(!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git) and 5(%load_ext nvcc_plugin) again to load the extensions. 
- Remember to not run cells 1,2 and 3 again. The following libraries are already installed on Google server that this notebook is using.
- cell 6 is there to check if the extension is working properly or not. If succesful then it will return a sorted array along with time taken by the program.

In [0]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-cg-toolkit' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-390' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-410' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-415' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-418' for glob 'nvidia*'
Note, selecting 'nvidi

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2020-05-07 07:52:33--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?bysXNcl-mgaA_bTEVjnm8kg2Q5IEkqZVeGZZB-Kl5JRU_k8xyKFd9gaxYhGLvoxyqyp1C2IfZ8DTWpaqgoQltTxOlgYZcMu2xggzPCnOoX8-y3olWEMy41r6OZjEupyt4cVBdhiA1bonKgrNdHq8SoEnRMZTau9in0n03qeCt6oLND7E-UAQgBBFTtu8oAaYEGCO3scC4BodmrMT-BU [following]
--2020-05-07 07:52:33--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?bysXNcl-mgaA_bTEVjnm8kg2Q5IEkqZVeGZZB-Kl5JRU_k8xyKFd9gaxYhGLvoxyqyp1C2IfZ8DTWpaqgoQltTxOlgYZcMu2xggzPCnOoX8-y3olWEMy41r6OZ

In [0]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-hupap0ld
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-hupap0ld
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=1837ac06c4a8793ebfa01c2ae45d21bd2086f01abe885bbe4d137e2a00a85804
  Stored in directory: /tmp/pip-ephem-wheel-cache-3ph_g2s1/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [0]:
%%cu
#include <bits/stdc++.h>
#include <omp.h> 
#include <stdlib.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <unistd.h>
using namespace std;
using namespace std::chrono; 

vector<int> arr;
const int mod = 1e9+7;

void prn(int * arr, int n){
    for(int i=0 ; i < n ; i++) cout<<arr[i]<<" ";cout<<endl;
}
__global__ void bitonic_sort(int* arr, int j, int k)
{
    int idx = threadIdx.x + blockDim.x * blockIdx.x;
    int i2 = idx ^ j;
    if(i2 > idx)
    {
        if((idx&k)==0)
        {
            if(arr[idx]>arr[i2])
            {
                int temp = arr[idx];
             arr[idx] = arr[i2];
             arr[i2] = temp;
            }
        }
        else{
            if(arr[idx]<arr[i2])
            {
                     int temp = arr[idx];
             arr[idx] = arr[i2];
             arr[i2] = temp;
           
            }
        }
    }
}
int main()
{
    int n= 512*512;
    int next = pow(2, ceil(log(n)/log(2)));
    srand(time(NULL));
    int *cuda_arr;
    int *arr = (int*)malloc(sizeof(int)*next);
    for(int i = 0 ; i < n ; i++)
    {
        arr[i] = rand()%mod;
    }
    for(int i = n ; i < next ; i++)
    {
        arr[i] = 0;
    }
    n = next;
    clock_t start = clock();
    cudaMalloc((void**) &cuda_arr, sizeof(int)*next);
    cudaMemcpy(cuda_arr, arr, sizeof(int)*next, cudaMemcpyHostToDevice);
    
    int BLOCKS = n / 4;
    int THREADS = 4;
    dim3 blocks(BLOCKS,1);    
    dim3 threads(THREADS,1);  
    int j, k;
    for (k = 2; k <= next; k <<= 1) {
        for (j=k>>1; j>0; j=j>>1) {
        bitonic_sort<<<blocks, threads>>>(cuda_arr, j, k);
        }
    }
    cudaMemcpy(arr, cuda_arr, sizeof(int)*next, cudaMemcpyDeviceToHost);
    cudaFree(cuda_arr);
    clock_t end = clock();
 double cpu_time_used = ((double) (end - start)) / CLOCKS_PER_SEC;
    cout<<"Time taken by cuda program : "<<cpu_time_used<<endl;
    prn(arr, n);
}

# Convulational Network (FFT based approach)
-  NOTE: if any other modular code with more convolution layers are available then add them here.

In [0]:
!pip install python-mnist

In [0]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4536490 sha256=7286d04d20478e9924b69d548d92c96510c84b311b67ca8c28ef667c6618bfcd
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2020.1-py2.py3-none-any.whl size=59602 sha256=71a1c0a49e19c9e7e8d316b75543791cf696f8f300e76eddf21a4c118ad82678
  Stored in directory: /root/.cache/pip/wheels/6f/da/1b/946775a88291378182ed92c9800d6d0ebc2a554cb89829cc24
Successfully built pycuda pytools


In [0]:
# All the imports 
import numpy as np
from keras.datasets import mnist
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [0]:
'''
MaxPooling code segment 
'''
class MaxPool2:
  # A Max Pooling layer using a pool size of 2.

  def iterate_regions(self, image):
    '''
    Generates non-overlapping 2x2 image regions to pool over.
    - image is a 2d numpy array
    '''
    h, w, _ = image.shape
    new_h = h // 2
    new_w = w // 2

    for i in range(new_h):
      for j in range(new_w):
        im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
        yield im_region, i, j

  def forward(self, input):
    '''
    Performs a forward pass of the maxpool layer using the given input.
    Returns a 3d numpy array with dimensions (h / 2, w / 2, num_filters).
    - input is a 3d numpy array with dimensions (h, w, num_filters)
    '''
    self.last_input = input

    h, w, num_filters = input.shape
    output = np.zeros((h // 2, w // 2, num_filters))

    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.amax(im_region, axis=(0, 1))

    return output

  def backprop(self, d_L_d_out):
    '''
    Performs a backward pass of the maxpool layer.
    Returns the loss gradient for this layer's inputs.
    - d_L_d_out is the loss gradient for this layer's outputs.
    '''
    d_L_d_input = np.zeros(self.last_input.shape)

    for im_region, i, j in self.iterate_regions(self.last_input):
      h, w, f = im_region.shape
      amax = np.amax(im_region, axis=(0, 1))

      for i2 in range(h):
        for j2 in range(w):
          for f2 in range(f):
            # If this pixel was the max value, copy the gradient to it.
            if im_region[i2, j2, f2] == amax[f2]:
              d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]

    return d_L_d_input

In [0]:
'''
Code segment for softmax
'''
class Softmax:
  # A standard fully-connected layer with softmax activation.

  def __init__(self, input_len, nodes):
    # We divide by input_len to reduce the variance of our initial values
    self.weights = np.random.randn(input_len, nodes) / input_len
    self.biases = np.zeros(nodes)

  def forward(self, input):
    '''
    Performs a forward pass of the softmax layer using the given input.
    Returns a 1d numpy array containing the respective probability values.
    - input can be any array with any dimensions.
    '''
    self.last_input_shape = input.shape

    input = input.flatten()
    self.last_input = input

    input_len, nodes = self.weights.shape

    totals = np.dot(input, self.weights) + self.biases
    self.last_totals = totals

    exp = np.exp(totals)
    return exp / np.sum(exp, axis=0)

  def backprop(self, d_L_d_out, learn_rate):
    '''
    Performs a backward pass of the softmax layer.
    Returns the loss gradient for this layer's inputs.
    - d_L_d_out is the loss gradient for this layer's outputs.
    - learn_rate is a double.
    '''
    # We know only 1 element of d_L_d_out will be nonzero
    for i, gradient in enumerate(d_L_d_out):
      if gradient == 0:
        continue

      # e^totals
      t_exp = np.exp(self.last_totals)

      # Sum of all e^totals
      S = np.sum(t_exp)

      # Gradients of out[i] against totals
      d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
      d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)

      # Gradients of totals against weights/biases/input
      d_t_d_w = self.last_input
      d_t_d_b = 1
      d_t_d_inputs = self.weights

      # Gradients of loss against totals
      d_L_d_t = gradient * d_out_d_t

      # Gradients of loss against weights/biases/input
      d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
      d_L_d_b = d_L_d_t * d_t_d_b
      d_L_d_inputs = d_t_d_inputs @ d_L_d_t

      # Update weights / biases
      self.weights -= learn_rate * d_L_d_w
      self.biases -= learn_rate * d_L_d_b

      return d_L_d_inputs.reshape(self.last_input_shape)

# Convolution Code
- We have two versions here.One direct using numpy and one using cuda and FFT(under work).

In [0]:
'''
Note: In this implementation, we assume the input is a 2d numpy array for simplicity, because that's
how our MNIST images are stored. This works for us because we use it as the first layer in our
network, but most CNNs have many more Conv layers. If we were building a bigger network that needed
to use Conv3x3 multiple times, we'd have to make the input be a 3d numpy array.
'''

class Conv3x3:
  # A Convolution layer using 3x3 filters.

  def __init__(self, num_filters):
    self.num_filters = num_filters

    # filters is a 3d array with dimensions (num_filters, 3, 3)
    # We divide by 9 to reduce the variance of our initial values
    self.filters = np.random.randn(num_filters, 3, 3) / 9

  def iterate_regions(self, image):
    '''
    Generates all possible 3x3 image regions using valid padding.
    - image is a 2d numpy array.
    '''
    h, w = image.shape

    for i in range(h - 2):
      for j in range(w - 2):
        im_region = image[i:(i + 3), j:(j + 3)]
        yield im_region, i, j

  def forward(self, input):
    '''
    Performs a forward pass of the conv layer using the given input.
    Returns a 3d numpy array with dimensions (h, w, num_filters).
    - input is a 2d numpy array
    '''
    self.last_input = input

    h, w = input.shape
    output = np.zeros((h - 2, w - 2, self.num_filters))

    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))

    return output

  def backprop(self, d_L_d_out, learn_rate):
    '''
    Performs a backward pass of the conv layer.
    - d_L_d_out is the loss gradient for this layer's outputs.
    - learn_rate is a float.
    '''
    d_L_d_filters = np.zeros(self.filters.shape)

    for im_region, i, j in self.iterate_regions(self.last_input):
      for f in range(self.num_filters):
        d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region

    # Update filters
    self.filters -= learn_rate * d_L_d_filters

    # We aren't returning anything here since we use Conv3x3 as the first layer in our CNN.
    # Otherwise, we'd need to return the loss gradient for this layer's inputs, just like every
    # other layer in our CNN.
    return None

# FFT version on convolution writtern in Cuda


In [0]:
mod = SourceModule("""
#include<stdio.h>
__global__ void inplace_divide_invert(double *A_r,double *A_i, double *B_r,double *B_i, int n, int threads)
{
    int i = blockIdx.x * threads + threadIdx.x;
    if (i < n)
    {
        // printf("in divide");
        B_r[i] /= n;
        B_i[i] /= n;
        A_r[i] = B_r[i];
        A_i[i] = B_i[i];
    }
}
__global__ void bitrev_reorder(double *__restrict__ r_r,double *__restrict__ r_i, double *__restrict__ d_r, double *__restrict__ d_i, int s, int nthr, int n)
{
    int id = blockIdx.x * nthr + threadIdx.x;
    int nwid = __brev(id) >> (32-s);
    if (id < n && nwid < n)
    {
        r_i[nwid] = d_i[id];
        r_r[nwid] = d_r[id];
    }
}
__device__ void inplace_fft_inner(double * A_r, double * A_i, double * B_r, double * B_i , int i, int j, int len, int n, int invert)
{
    if ((i + j + (len / 2)) < n && j < (len / 2))
    {
        double u_r, v_r;
        double u_i, v_i;
        double angle = (2 * 3.14 * j) / (len * (invert ? 1.0 : -1.0));
        v_r = __cosf(angle);
        v_i = __sinf(angle);
        u_r = B_r[i + j];
        u_i = B_i[i + j];
        double temp_vr = v_r,temp_vi = v_i;
        v_r = B_r[i + j + (len / 2)]*temp_vr - B_i[i + j + (len / 2)]*temp_vi;
        v_i = B_i[i + j + (len / 2)]*temp_vr + B_r[i + j + (len / 2)]*temp_vi;
        A_r[i + j] = u_r + v_r;
        A_i[i + j] = u_i + v_i;
        A_r[i + j + len / 2] = u_r - v_r;
        A_i[i + j + len / 2] = u_i - v_i;
        B_r[i + j] = A_r[i + j];
        B_i[i + j] = A_i[i + j];
        B_r[i + j + len / 2] = A_r[i + j + len / 2];
        B_i[i + j + len / 2] = A_i[i + j + len / 2];
    }
}
__global__ void inplace_fft(double *__restrict__ A_r, double *__restrict__ A_i,double *__restrict__ B_r, double *__restrict__ B_i , int i, int len, int n, int threads, int invert)
{
    int j = blockIdx.x * threads + threadIdx.x;
    inplace_fft_inner(A_r, A_i, B_r, B_i, i, j, len, n, invert);
    
}
__global__ void inplace_fft_outer(double *__restrict__ A_r, double *__restrict__ A_i, double *__restrict__ B_r, double *__restrict__ B_i , int len, int n, int threads, int invert)
{
    int i = (blockIdx.x * threads + threadIdx.x)*len;
    for (int j = 0; j < len / 2; j++)
    {
        inplace_fft_inner(A_r, A_i, B_r, B_i, i, j, len, n, invert);
    }
}
""")

In [0]:
bitrev_reorder = mod.get_function("bitrev_reorder")
inplace_fft_outer = mod.get_function("inplace_fft_outer") 
inplace_fft = mod.get_function("inplace_fft")
inplace_divide_invert = mod.get_function("inplace_divide_invert")

In [0]:
def fft(row, invert, balance, threads):
    n = float(row.shape[0])
    row = row.astype(np.double)
    zrs = np.zeros(row.shape).astype(np.double)
    fin_r = np.empty_like(row).astype(np.double)
    fin_i = np.empty_like(row).astype(np.double)

    s = np.int32(np.log2(row.shape[0]))
    blksz = int(np.ceil(n / threads))
    bitrev_reorder(cuda.Out(fin_r), cuda.Out(fin_i), cuda.In(row), cuda.In(zrs), np.int32(s), np.int32(threads), np.int32(n),grid = (blksz, 1), block = (int(threads), 1, 1))
    print(fin_r)
    cuda.Context.synchronize()
    # print("Synchronized")
    ln = 2
    ln = int(ln)
    while ln <= n:
        if n // ln > balance:
            blksz = int(np.ceil(n / threads / ln))
            inplace_fft_outer(cuda.Out(fin_r), cuda.Out(fin_i), np.int32(ln), np.int32(n), np.int32(threads), np.int32(invert),grid = (blksz, 1), block = (threads, 1, 1))
        else:
            for i in range(0, int(n), ln):
                rep = float(ln) / 2
                rep = int(np.ceil(rep / threads))
                temp_r = np.copy(fin_r).astype(np.double)
                temp_i = np.copy(fin_i).astype(np.double)
                temp = inplace_fft(cuda.Out(fin_r), cuda.Out(fin_i), cuda.In(temp_r), cuda.In(temp_i), np.int32(i), np.int32(ln), np.int32(n), np.int32(threads), np.int32(invert),grid = (rep, 1), block = (threads, 1, 1))
        ln = ln * 2 
    if invert==1:
        blksz = int(np.ceil(n / threads))
        temp_r = np.copy(fin_r).astype(np.double32)
        temp_i = np.copy(fin_i).astype(np.double32)
        inplace_divide_invert(cuda.Out(fin_r), cuda.Out(fin_i), cuda.In(temp_r), cuda.In(temp_i), np.int32(n), np.int32(threads),grid = (blksz, 1), block = (threads, 1, 1))
    return fin_r


def fft2(inp, invert, balance, threads):
    img = inp.copy()
    shp = img.shape
    for ind, key in enumerate(img):
        img[ind] = fft(key, invert, balance, threads)
    img = np.transpose(img)
    for ind,key in enumerate(img):
        img[ind] = fft(key, invert, balance, threads)
    return np.transpose(img)


In [0]:
'''
Note: In this implementation, we assume the input is a 2d numpy array for simplicity, because that's
how our MNIST images are stored. This works for us because we use it as the first layer in our
network, but most CNNs have many more Conv layers. If we were building a bigger network that needed
to use Conv3x3 multiple times, we'd have to make the input be a 3d numpy array.
'''

class ConvFFT:
  # A Convolution layer using 3x3 filters.

  def __init__(self, num_filters, sz):
    self.num_filters = num_filters

    # filters is a 3d array with dimensions (num_filters, 3, 3)
    # We divide by 9 to reduce the variance of our initial values
    self.filters = np.random.randn(num_filters, sz, sz) / (sz*sz)
    self.filter_fft = fft2(self.filters, 0, 1024, 100)
  def forward(self, input):
    '''
    Performs a forward pass of the conv layer using the given input.
    Returns a 3d numpy array with dimensions (h, w, num_filters).
    - input is a 2d numpy array
    '''
    self.last_input = input

    # h, w = input.shape
    # output = np.zeros((h - 2, w - 2, self.num_filters))

    # for im_region, i, j in self.iterate_regions(input):
    #   output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))

    # return output


  def backprop(self, d_L_d_out, learn_rate):
    '''
    Performs a backward pass of the conv layer.
    - d_L_d_out is the loss gradient for this layer's outputs.
    - learn_rate is a double.
    '''
    d_L_d_filters = np.zeros(self.filters.shape)

    for im_region, i, j in self.iterate_regions(self.last_input):
      for f in range(self.num_filters):
        d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region

    # Update filters
    self.filters -= learn_rate * d_L_d_filters

    # We aren't returning anything here since we use Conv3x3 as the first layer in our CNN.
    # Otherwise, we'd need to return the loss gradient for this layer's inputs, just like every
    # other layer in our CNN.
    return None

In [0]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images[:2000]
train_labels = train_labels[:2000]
test_images  = test_images[:2000]
test_labels  = test_labels[:2000]
conv = Conv3x3(8)                  # 28x28x1 -> 26x26x8
pool = MaxPool2()                  # 26x26x8 -> 13x13x8
softmax = Softmax(13 * 13 * 8, 10) # 13x13x8 -> 10

def forward(image, label):
  '''
  Completes a forward pass of the CNN and calculates the accuracy and
  cross-entropy loss.
  - image is a 2d numpy array
  - label is a digit
  '''
  # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
  # to work with. This is standard practice.
  out = conv.forward((image / 255) - 0.5)
  out = pool.forward(out)
  out = softmax.forward(out)

  # Calculate cross-entropy loss and accuracy. np.log() is the natural log.
  loss = -np.log(out[label])
  acc = 1 if np.argmax(out) == label else 0

  return out, loss, acc

def train(im, label, lr=.005):
  '''
  Completes a full training step on the given image and label.
  Returns the cross-entropy loss and accuracy.
  - image is a 2d numpy array
  - label is a digit
  - lr is the learning rate
  '''
  # Forward
  out, loss, acc = forward(im, label)

  # Calculate initial gradient
  gradient = np.zeros(10)
  gradient[label] = -1 / out[label]

  # Backprop
  gradient = softmax.backprop(gradient, lr)
  gradient = pool.backprop(gradient)
  gradient = conv.backprop(gradient, lr)

  return loss, acc

print('MNIST CNN initialized!')

# Train the CNN for 3 epochs
for epoch in range(3):
  print('--- Epoch %d ---' % (epoch + 1))

  # Shuffle the training data
  permutation = np.random.permutation(len(train_images))
  train_images = train_images[permutation]
  train_labels = train_labels[permutation]

  # Train!
  loss = 0
  num_correct = 0
  for i, (im, label) in enumerate(zip(train_images, train_labels)):
    if i % 100 == 99:
      print(
        '[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %
        (i + 1, loss / 100, num_correct)
      )
      loss = 0
      num_correct = 0

    l, acc = train(im, label)
    loss += l
    num_correct += acc

# Test the CNN
print('\n--- Testing the CNN ---')
loss = 0
num_correct = 0
for im, label in zip(test_images, test_labels):
  _, l, acc = forward(im, label)
  loss += l
  num_correct += acc

num_tests = len(test_images)
print('Test Loss:', loss / num_tests)
print('Test Accuracy:', num_correct / num_tests)

11493376/11490434 [==============================] - 2s 0us/step
MNIST CNN initialized!
--- Epoch 1 ---
[Step 100] Past 100 steps: Average Loss 2.261 | Accuracy: 15%
[Step 200] Past 100 steps: Average Loss 2.079 | Accuracy: 30%
[Step 300] Past 100 steps: Average Loss 1.730 | Accuracy: 51%
[Step 400] Past 100 steps: Average Loss 1.352 | Accuracy: 57%
[Step 500] Past 100 steps: Average Loss 1.060 | Accuracy: 64%
[Step 600] Past 100 steps: Average Loss 0.825 | Accuracy: 75%
[Step 700] Past 100 steps: Average Loss 0.791 | Accuracy: 75%
[Step 800] Past 100 steps: Average Loss 0.593 | Accuracy: 81%
[Step 900] Past 100 steps: Average Loss 0.863 | Accuracy: 73%
[Step 1000] Past 100 steps: Average Loss 0.685 | Accuracy: 77%
[Step 1100] Past 100 steps: Average Loss 0.624 | Accuracy: 76%
[Step 1200] Past 100 steps: Average Loss 0.627 | Accuracy: 81%
[Step 1300] Past 100 steps: Average Loss 0.492 | Accuracy: 85%
[Step 1400] Past 100 steps: Average Loss 0.640 | Accuracy: 79%
[Step 1500] Past 100 st